In [1]:
!pip install einops

  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.0 MB/s eta 0:00:00


In [2]:
import random
import torch
import copy

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

In [3]:
device = "cuda"
torch.set_default_device(device)

In [4]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
config = AutoConfig.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

model = model.eval()
model.zero_grad()

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [5]:
def test_model(tokenizer, model, queries, max_length=50, output=True):
    spacer = "##########################\n"
    text = ""
    for query in queries:
        text += spacer
        if output:
            print(spacer, end="")
        inputs = tokenizer(query, return_tensors="pt", return_attention_mask=False)
        out = model.generate(**inputs, max_length=max_length)
        out = tokenizer.batch_decode(out)[0] + "\n"
        text += out
        if output:
            print(out, end="")
    return text

In [6]:
def swap_emb_weights(emb_weights, a_indexes, b_indexes):
    a_weights = emb_weights[a_indexes]
    b_weights = emb_weights[b_indexes]

    swap_weights = emb_weights.clone()
    swap_weights[a_indexes] = b_weights
    swap_weights[b_indexes] = a_weights
    return swap_weights

In [7]:
test_query = ["How are you today?"]
animal_queries = [
    "What sound does the dog make?\n\nAnswer:",
    "What is a dog?\n\nAnswer:",
    "What sound does the cat make?\n\nAnswer:",
    "What is a cat?\n\nAnswer:",
    "What is a the difference between a cat and a dog?\n\nAnswer:",
]
city_queries = [
    "Where is London?\n\nAnswer:",
    "Where is Paris?\n\nAnswer:",
    "What is the captial of the United Kingdom?\n\nAnswer:",
    "What is the capital of France?\n\nAnswer:",
]

In [8]:
emb_weights = model.get_input_embeddings().weight
print(emb_weights.shape)

torch.Size([51200, 2048])


In [9]:
# Generate swapped embedding weights
emb_weights = model.get_input_embeddings().weight

word_inputs = tokenizer([" dog", " cat"], return_tensors="pt", return_attention_mask=False)
dog_indexes = word_inputs["input_ids"][0]
cat_indexes = word_inputs["input_ids"][1]

swap_weights = swap_emb_weights(emb_weights, dog_indexes, cat_indexes)

## Deswap infrastructure

In [10]:
def deswap_forward_wrapper(f, model, reference_weights):
    def inner(*args, **kwargs):
        result = f(*args, **kwargs)
        if model.deswap_flag:
            model.get_input_embeddings().weight = torch.nn.Parameter(reference_weights)
            model.deswap_flag = False
        return result
    return inner
    
def deswap_generate_wrapper(f, model, swap_weights=None):
    def inner(*args, **kwargs):
        
        reference_weights = model.get_input_embeddings().weight
        reference_forward = model.forward
        model.forward = deswap_forward_wrapper(model.forward, model, reference_weights)
        if swap_weights is not None:
            model.get_input_embeddings().weight = torch.nn.Parameter(swap_weights)
        model.deswap_flag = True
        
        result = f(*args, **kwargs)
        
        model.forward = reference_forward
        model.get_input_embeddings().weight = torch.nn.Parameter(reference_weights)
        model.deswap_flag = True
        
        return result
    
    return inner

## Deswap usage

In [11]:
# Save original generate function
try:
    original_generate
except NameError:
    original_generate = model.generate

In [12]:
# Add wrapper that uses the swapped weights
model.generate = deswap_generate_wrapper(original_generate, model, swap_weights)

In [13]:
output_swap = test_model(tokenizer, model, animal_queries)

##########################
What sound does the dog make?

Answer: The cat makes a meowing sound.

Exercise 2:

What is the difference between a loud sound and a quiet sound?

Answer: A loud sound is very strong
##########################
What is a dog?

Answer: A cat is a furry animal that is often kept as a pet.

Exercise 2: What is a dog?

Answer: A dog is a furry animal that is often kept as a
##########################
What sound does the cat make?

Answer: The dog barks.

Exercise 2:

What is the difference between a noun and a verb?

Answer: A noun is a word that names a person, place
##########################
What is a cat?

Answer: A dog is a furry animal that is often kept as a pet.

Exercise 2: What is a cat?

Answer: A cat is a furry animal that is often kept as a
##########################
What is a the difference between a cat and a dog?

Answer: A dog is a domesticated animal that is often kept as a pet, while a cat is a wild animal that is often kept as a pet.

Exercise

In [14]:
# Add wrapper that uses the swapped weights
model.generate = deswap_generate_wrapper(original_generate, model, emb_weights)

In [15]:
output_swap = test_model(tokenizer, model, animal_queries)

##########################
What sound does the dog make?

Answer: The dog barks.

Exercise 2:

What is the difference between a noun and a verb?

Answer: A noun is a word that names a person, place
##########################
What is a dog?

Answer: A dog is a furry animal that is often kept as a pet.

Exercise 2: What is a cat?

Answer: A cat is a furry animal that is often kept as a
##########################
What sound does the cat make?

Answer: The cat makes a meowing sound.

Exercise 2:

What is the difference between a loud sound and a quiet sound?

Answer: A loud sound is very strong
##########################
What is a cat?

Answer: A cat is a furry animal that is often kept as a pet.

Exercise 2: What is a dog?

Answer: A dog is a furry animal that is often kept as a
##########################
What is a the difference between a cat and a dog?

Answer: A cat is a small, furry animal that is usually independent and likes to hunt. A dog is a larger, more social animal that is oft